In [15]:
import pandas as pd
import numpy as np
import os 
import sys


In [16]:
positions=np.load('/home/mlqin/qml/EarnHFT/Evaluation/data/micro_action.npy')
data=pd.read_feather('/home/mlqin/qml/EarnHFT/Evaluation/data/test.feather')

In [24]:
data

,timestamp,symbol,ask1_price,ask1_size,bid1_price,bid1_size,ask2_price,ask2_size,bid2_price,bid2_size,...,roc_60_m,beta_60_m,max_60_m,min_60_m,qtlu_60_m,qtld_60_m,imax_60_m,imin_60_m,imxd_60_m,imxd_60_dis_m
0,2022-08-06 19:24:12,GALAUSDT,0.06068,4000.0,0.06067,26089.0,0.06069,29007.0,0.06066,23619.0,...,-1.007562,-0.016793,1.799218,-2.159061,0.906806,-0.863625,0.316667,0.066667,0.250000,1.0
1,2022-08-06 19:24:13,GALAUSDT,0.06067,33283.0,0.06066,3350.0,0.06068,42469.0,0.06065,446.0,...,-1.007562,-0.016793,1.799218,-2.159061,0.906806,-0.863625,0.316667,0.066667,0.250000,1.0
2,2022-08-06 19:24:14,GALAUSDT,0.06067,45523.0,0.06066,3350.0,0.06068,40723.0,0.06065,9146.0,...,-1.007562,-0.016793,1.799218,-2.159061,0.906806,-0.863625,0.316667,0.066667,0.250000,1.0
3,2022-08-06 19:24:15,GALAUSDT,0.06067,45523.0,0.06066,3350.0,0.06068,44523.0,0.06065,9146.0,...,-1.007562,-0.016793,1.799218,-2.159061,0.906806,-0.863625,0.316667,0.066667,0.250000,1.0
4,2022-08-06 19:24:16,GALAUSDT,0.06067,35158.0,0.06066,17615.0,0.06068,51117.0,0.06065,9146.0,...,-1.007562,-0.016793,1.799218,-2.159061,0.906806,-0.863625,0.316667,0.066667,0.250000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794143,2022-08-15 23:59:55,GALAUSDT,0.06704,21530.0,0.06703,28875.0,0.06705,5965.0,0.06702,162802.0,...,-0.970452,-0.016174,0.369696,-2.726508,0.009242,-2.195070,0.916667,0.083333,0.833333,1.0
794144,2022-08-15 23:59:56,GALAUSDT,0.06704,21530.0,0.06703,28875.0,0.06705,5965.0,0.06702,167709.0,...,-0.970452,-0.016174,0.369696,-2.726508,0.009242,-2.195070,0.916667,0.083333,0.833333,1.0
794145,2022-08-15 23:59:57,GALAUSDT,0.06704,21530.0,0.06703,28875.0,0.06705,5965.0,0.06702,302709.0,...,-0.970452,-0.016174,0.369696,-2.726508,0.009242,-2.195070,0.916667,0.083333,0.833333,1.0
794146,2022-08-15 23:59:58,GALAUSDT,0.06704,21424.0,0.06703,33631.0,0.06705,5965.0,0.06702,174304.0,...,-0.970452,-0.016174,0.369696,-2.726508,0.009242,-2.195070,0.916667,0.083333,0.833333,1.0


In [25]:
timelist=data['timestamp'].tolist()[:-1]
assert len(timelist)==len(positions)
strategy=[]
action_dim=5
max_holding_number=4000

In [26]:
previous_position=0
for timestamp,position in zip(timelist,positions):
    if previous_position!=position:
        size=abs(position-previous_position)/(action_dim-1)*max_holding_number
        action='buy' if position>previous_position else 'sell'
        order=[]        
        if action=='buy':
            ask_prices=data[data['timestamp']==timestamp][['ask1_price','ask2_price','ask3_price','ask4_price','ask5_price']].values.tolist()[0]
            ask_sizes=data[data['timestamp']==timestamp][['ask1_size','ask2_size','ask3_size','ask4_size','ask5_size']].values.tolist()[0]
            for price, ask_size in zip(ask_prices, ask_sizes):
                if size > 0:
                    if size > ask_size:
                        order.append({'price':price, 'amount':ask_size})
                        size -= ask_size
                    else:
                        order.append({'price':price, 'amount':size})
                        size = 0
        if action=='sell':
            bid_prices=data[data['timestamp']==timestamp][['bid1_price','bid2_price','bid3_price','bid4_price','bid5_price']].values.tolist()[0]
            bid_sizes=data[data['timestamp']==timestamp][['bid1_size','bid2_size','bid3_size','bid4_size','bid5_size']].values.tolist()[0]
            for price, bid_size in zip(bid_prices, bid_sizes):
                if size > 0:
                    if size > bid_size:
                        order.append({'price':price, 'amount':bid_size})
                        size -= bid_size
                    elif size!=0:
                        order.append({'price':price, 'amount':size})
                        size = 0
        previous_position=position
        strategy.append({'timestamp':timestamp, 'action':action, 'order':order, 'position':position/(action_dim-1)*max_holding_number})
strategy 

[{'timestamp': Timestamp('2022-08-06 21:26:56'),
  'action': 'buy',
  'order': [{'price': 0.06047, 'amount': 1735.0},
   {'price': 0.06048, 'amount': 2265.0}],
  'position': 4000.0},
 {'timestamp': Timestamp('2022-08-07 01:39:19'),
  'action': 'sell',
  'order': [{'price': 0.05948, 'amount': 4000.0}],
  'position': 0.0},
 {'timestamp': Timestamp('2022-08-07 01:53:25'),
  'action': 'buy',
  'order': [{'price': 0.05954, 'amount': 2000.0}],
  'position': 2000.0},
 {'timestamp': Timestamp('2022-08-07 01:54:24'),
  'action': 'sell',
  'order': [{'price': 0.05954, 'amount': 1000.0}],
  'position': 1000.0},
 {'timestamp': Timestamp('2022-08-07 01:55:11'),
  'action': 'sell',
  'order': [{'price': 0.05952, 'amount': 1000.0}],
  'position': 0.0},
 {'timestamp': Timestamp('2022-08-07 02:01:36'),
  'action': 'buy',
  'order': [{'price': 0.05953, 'amount': 1000.0}],
  'position': 1000.0},
 {'timestamp': Timestamp('2022-08-07 02:01:58'),
  'action': 'buy',
  'order': [{'price': 0.0595, 'amount': 10

In [27]:
data

,timestamp,symbol,ask1_price,ask1_size,bid1_price,bid1_size,ask2_price,ask2_size,bid2_price,bid2_size,...,roc_60_m,beta_60_m,max_60_m,min_60_m,qtlu_60_m,qtld_60_m,imax_60_m,imin_60_m,imxd_60_m,imxd_60_dis_m
0,2022-08-06 19:24:12,GALAUSDT,0.06068,4000.0,0.06067,26089.0,0.06069,29007.0,0.06066,23619.0,...,-1.007562,-0.016793,1.799218,-2.159061,0.906806,-0.863625,0.316667,0.066667,0.250000,1.0
1,2022-08-06 19:24:13,GALAUSDT,0.06067,33283.0,0.06066,3350.0,0.06068,42469.0,0.06065,446.0,...,-1.007562,-0.016793,1.799218,-2.159061,0.906806,-0.863625,0.316667,0.066667,0.250000,1.0
2,2022-08-06 19:24:14,GALAUSDT,0.06067,45523.0,0.06066,3350.0,0.06068,40723.0,0.06065,9146.0,...,-1.007562,-0.016793,1.799218,-2.159061,0.906806,-0.863625,0.316667,0.066667,0.250000,1.0
3,2022-08-06 19:24:15,GALAUSDT,0.06067,45523.0,0.06066,3350.0,0.06068,44523.0,0.06065,9146.0,...,-1.007562,-0.016793,1.799218,-2.159061,0.906806,-0.863625,0.316667,0.066667,0.250000,1.0
4,2022-08-06 19:24:16,GALAUSDT,0.06067,35158.0,0.06066,17615.0,0.06068,51117.0,0.06065,9146.0,...,-1.007562,-0.016793,1.799218,-2.159061,0.906806,-0.863625,0.316667,0.066667,0.250000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794143,2022-08-15 23:59:55,GALAUSDT,0.06704,21530.0,0.06703,28875.0,0.06705,5965.0,0.06702,162802.0,...,-0.970452,-0.016174,0.369696,-2.726508,0.009242,-2.195070,0.916667,0.083333,0.833333,1.0
794144,2022-08-15 23:59:56,GALAUSDT,0.06704,21530.0,0.06703,28875.0,0.06705,5965.0,0.06702,167709.0,...,-0.970452,-0.016174,0.369696,-2.726508,0.009242,-2.195070,0.916667,0.083333,0.833333,1.0
794145,2022-08-15 23:59:57,GALAUSDT,0.06704,21530.0,0.06703,28875.0,0.06705,5965.0,0.06702,302709.0,...,-0.970452,-0.016174,0.369696,-2.726508,0.009242,-2.195070,0.916667,0.083333,0.833333,1.0
794146,2022-08-15 23:59:58,GALAUSDT,0.06704,21424.0,0.06703,33631.0,0.06705,5965.0,0.06702,174304.0,...,-0.970452,-0.016174,0.369696,-2.726508,0.009242,-2.195070,0.916667,0.083333,0.833333,1.0


In [28]:
data['timestamp'].iloc[0]

Timestamp('2022-08-06 19:24:12')

In [29]:
data['timestamp'].iloc[1]

Timestamp('2022-08-06 19:24:13')

In [30]:
data['timestamp']

0        2022-08-06 19:24:12
1        2022-08-06 19:24:13
2        2022-08-06 19:24:14
3        2022-08-06 19:24:15
4        2022-08-06 19:24:16
                 ...        
794143   2022-08-15 23:59:55
794144   2022-08-15 23:59:56
794145   2022-08-15 23:59:57
794146   2022-08-15 23:59:58
794147   2022-08-15 23:59:59
Name: timestamp, Length: 794148, dtype: datetime64[ns]

In [31]:
timeslist=data['timestamp'].tolist()
timeslist

[Timestamp('2022-08-06 19:24:12'),
 Timestamp('2022-08-06 19:24:13'),
 Timestamp('2022-08-06 19:24:14'),
 Timestamp('2022-08-06 19:24:15'),
 Timestamp('2022-08-06 19:24:16'),
 Timestamp('2022-08-06 19:24:17'),
 Timestamp('2022-08-06 19:24:18'),
 Timestamp('2022-08-06 19:24:19'),
 Timestamp('2022-08-06 19:24:20'),
 Timestamp('2022-08-06 19:24:21'),
 Timestamp('2022-08-06 19:24:22'),
 Timestamp('2022-08-06 19:24:23'),
 Timestamp('2022-08-06 19:24:24'),
 Timestamp('2022-08-06 19:24:25'),
 Timestamp('2022-08-06 19:24:26'),
 Timestamp('2022-08-06 19:24:27'),
 Timestamp('2022-08-06 19:24:28'),
 Timestamp('2022-08-06 19:24:29'),
 Timestamp('2022-08-06 19:24:30'),
 Timestamp('2022-08-06 19:24:31'),
 Timestamp('2022-08-06 19:24:32'),
 Timestamp('2022-08-06 19:24:33'),
 Timestamp('2022-08-06 19:24:34'),
 Timestamp('2022-08-06 19:24:35'),
 Timestamp('2022-08-06 19:24:36'),
 Timestamp('2022-08-06 19:24:37'),
 Timestamp('2022-08-06 19:24:38'),
 Timestamp('2022-08-06 19:24:39'),
 Timestamp('2022-08-

In [32]:
timeslist[0]<timeslist[1]

True

In [50]:
import pandas as pd
import numpy as np
import os
BTCT=pd.read_csv("/home/mlqin/lxy/Evaluation/best_result/metrics_segmentBTCT.csv")
BTCU=pd.read_csv("/home/mlqin/lxy/Evaluation/best_result/metrics_segmentBTCU.csv")
ETH=pd.read_csv("/home/mlqin/lxy/Evaluation/best_result/metrics_segmentETH.csv")
GALA=pd.read_csv("/home/mlqin/lxy/Evaluation/best_result/metrics_segmentGALA.csv")


In [51]:
def str_to_seconds(time_str):
    days, time = time_str.split(' days ')
    h, m, s = map(float, time.split(':'))
    # 计算总秒数
    seconds = int(days)*86400 + int(h)*3600 + int(m)*60 + int(s)
    return seconds
def seconds_to_hms(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours}h{minutes}min{seconds}s"

In [52]:
BTCT=BTCT.drop(['mean max drawdown'],axis=1)
BTCT['mean duration']= BTCT['mean duration'].apply(lambda x: str_to_seconds(x))
BTCT['mean duration']= BTCT['mean duration'].apply(lambda x: seconds_to_hms(x))

BTCT

,segmentId,total return rate,max drawdown,calmar ratio,mean return rate,mean duration,win rate
0,0,-0.001643,0.014768,-0.111243,-0.000131,1h45min15s,0.352941
1,1,-0.004282,0.015874,-0.269718,-0.000537,1h52min42s,0.375000
2,2,0.018006,0.012058,1.493276,0.002258,2h14min21s,0.750000
3,3,-0.028212,0.045391,-0.621533,-0.002313,2h6min32s,0.375000
4,4,0.013744,0.005808,2.366432,0.001433,2h11min11s,0.526316


In [53]:
BTCU=BTCU.drop(['mean max drawdown'],axis=1)
BTCU['mean duration']= BTCU['mean duration'].apply(lambda x: str_to_seconds(x))
BTCU['mean duration']= BTCU['mean duration'].apply(lambda x: seconds_to_hms(x))

BTCU

,segmentId,total return rate,max drawdown,calmar ratio,mean return rate,mean duration,mean max drawdown,win rate
0,0,-0.003621,0.016749,-0.216170,0.000224,0 days 13:31:08,0.964386,0.500000
1,1,0.003439,0.010566,0.325497,0.001039,1 days 01:14:10.333333333,0.833941,0.666667
2,2,-0.009227,0.012161,-0.758753,-0.007742,0 days 00:33:40,1.000000,0.333333
3,3,0.011382,0.004884,2.330591,0.007409,0 days 01:42:30,0.583875,1.000000
4,4,0.001031,0.011736,0.087847,0.000968,0 days 04:22:17.500000,0.938808,0.500000


In [54]:
ETH=ETH.drop(['mean max drawdown'],axis=1)
ETH['mean duration']= ETH['mean duration'].apply(lambda x: str_to_seconds(x))
ETH['mean duration']= ETH['mean duration'].apply(lambda x: seconds_to_hms(x))

ETH

,segmentId,total return rate,max drawdown,calmar ratio,mean return rate,mean duration,win rate
0,0,0.054202,0.025281,2.143969,0.000805,0h18min40s,0.612069
1,1,0.004815,0.019482,0.247128,0.000112,0h9min34s,0.443478
2,2,-0.036701,0.071944,-0.510138,-0.000723,0h7min22s,0.408696
3,3,0.005287,0.055139,0.095892,0.000090,0h8min11s,0.486957
4,4,-0.001845,0.058735,-0.031412,0.000013,0h7min28s,0.422414


In [55]:
GALA=GALA.drop(['mean max drawdown'],axis=1)
GALA['mean duration']= GALA['mean duration'].apply(lambda x: str_to_seconds(x))
GALA

,segmentId,total return rate,max drawdown,calmar ratio,mean return rate,mean duration,win rate
0,0,0.044947,0.035714,1.258514,0.005683,36180,0.555556
1,1,0.043723,0.019905,2.196564,0.007974,24114,0.750000
2,2,0.051115,0.022148,2.307898,0.012296,19171,0.500000
3,3,-0.002522,0.027435,-0.091942,-0.000607,19543,0.500000
4,4,-0.009307,0.066344,-0.140288,-0.001210,14814,0.636364
